# Assignment 2
## Task 1:
*Adjust the tutorial implementation to perform the given prediction task and perform a suitable evaluation on a dedicated test set.*

*For example, you can compute the Euclidean distance between the target point and the predicted point.*

---

#### Import packages


In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

>   - `pandas` is used to read and manipulate CSV files (your node and edge data).
>   - `numpy` is useful for numerical arrays and vectorized math operations.
>   - `tensorflow` & `keras` will be used to build and train the GAT model.
>   - `layers` gives access to Keras components like Dense layers, activation functions, etc.


#### Data Aggregation from Multiple Scenes

In [ ]:
# Directory containing .nodes and .edges files
DATASET_PATH = "dataset"

# Get all scene IDs (each scene = a file pair: .nodes + .edges)
scene_ids = sorted(set(f.split(".")[0] for f in os.listdir(DATASET_PATH)))

# Lists to store combined node and edge data
all_nodes = []
all_edges = []
offset = 0 # Offset to make node IDs unique across all scenes

for scene_id in scene_ids:
    nodes_path = os.path.join(DATASET_PATH, f"{scene_id}.nodes")
    edges_path = os.path.join(DATASET_PATH, f"{scene_id}.edges")
    
    if not os.path.exists(nodes_path) or not os.path.exists(edges_path):
        continue # Skip if files are missing
    
    # Load node attributes
    nodes_df = pd.read_csv(
        nodes_path, header=None,
        names=["node_id", "current_x", "current_y", "prev_x", "prev_y", "future_x", "future_y"],
        na_values=["_", "NA", "NaN", "nan"]
    )
    
    # Remove rows with missing position data (except for future positions)
    nodes_df = nodes_df.dropna(subset=["current_x", "current_y", "prev_x", "prev_y"])
    nodes_df[["current_x", "current_y", "prev_x", "prev_y"]] = nodes_df[["current_x", "current_y", "prev_x", "prev_y"]].astype(float)
    
    # Reindex local node IDs into global unique IDs
    local_to_global = {nid: i + offset for i, nid in enumerate(nodes_df["node_id"])}
    nodes_df["global_id"] = nodes_df["node_id"].map(local_to_global)
    
    # Load and remap edge list to global IDs
    edges_df = pd.read_csv(edges_path, header=None, names=["target", "source"])
    edges_df["source"] = edges_df["source"].map(local_to_global)
    edges_df["target"] = edges_df["target"].map(local_to_global)
    
    # Duplicate edges in the opposite direction to make graph undirected
    reversed_edges = edges_df.rename(columns={"source": "target", "target": "source"})
    full_edges = pd.concat([edges_df, reversed_edges])
    
    # Store the processed scene data
    all_nodes.append(nodes_df)
    all_edges.append(full_edges)

    # Update offset for next scene
    offset += len(nodes_df)
    
# Concatenate all scenes into single global DataFrames
nodes_all = pd.concat(all_nodes).sort_values("global_id")
edges_all = pd.concat(all_edges).dropna().astype(int)

>   - This block **loads and aggregates multiple scenes** into one global graph structure.  
>   - Node IDs are **reindexed globally** to avoid collisions between scenes.  
>   - Missing `current` or `previous` positions are filtered out.  
        `future_x/y` is kept (even if missing) because it’s only needed at **prediction time**.  
>   - Graph is made **undirected** by manually adding reverse edges.
>   - Final output:  
>       - `nodes_all`: contains all node features, cleaned and reindexed.  
>       - `edges_all`: contains all edges across all scenes, with global node IDs.

#### Tensor Creation for Model Input

In [ ]:
# Extract node input features as float tensor (shape: [num_nodes, 4])
node_features = tf.convert_to_tensor(
    nodes_all[["current_x", "current_y", "prev_x", "prev_y"]].to_numpy(),
    dtype=tf.float32
)

# Extract labels (future positions), replacing NaN with 0.0 to avoid tensor errors
labels = tf.convert_to_tensor(
    nodes_all[["future_x", "future_y"]].fillna(0.0).to_numpy(),
    dtype=tf.float32
)

# Create a mask to mark which nodes have valid future positions (used for loss filtering)
mask = tf.convert_to_tensor(
    ~nodes_all[["future_x", "future_y"]].isna().any(axis=1),
    dtype=tf.bool
)

# Create edge list as tensor (shape: [num_edges, 2]), each row: [target, source]
edges = tf.convert_to_tensor(
    edges_all[["target", "source"]].to_numpy(),
    dtype=tf.int64
)

>   - `node_features`: 4D input vector for each node (current + previous positions).
>   - `labels`: target (x, y) positions, with NaNs replaced by `0.0` to keep tensor shape consistent.
>   - `mask`: boolean mask **to filter valid training samples** (nodes with known future positions).
>   - `edges`: edge list formatted for a GNN — each edge connects two nodes using global IDs.

#### Train-Test Split

In [ ]:
# Shuffle all node indices and split 80/20 for training and testing
split_ratio = 0.8
random_indices = np.random.permutation(len(nodes_all)) # Randomly permuted indices
split_idx = int(split_ratio * len(random_indices))
train_indices = random_indices[: split_idx]
test_indices = random_indices[split_idx :]

# Select node features and labels using the indices (converted to NumPy first)
train_node_features = node_features.numpy()[train_indices]
test_node_features = node_features.numpy()[test_indices]

train_labels = labels.numpy()[train_indices]
test_labels = labels.numpy()[test_indices]

train_mask = mask.numpy()[train_indices]
test_mask = mask.numpy()[test_indices]

# Convert everything back to TensorFlow tensors
train_node_features = tf.convert_to_tensor(train_node_features, dtype=tf.float32)
test_node_features = tf.convert_to_tensor(test_node_features, dtype=tf.float32)

train_labels = tf.convert_to_tensor(train_labels, dtype=tf.float32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.float32)

train_mask = tf.convert_to_tensor(train_mask, dtype=tf.bool)
test_mask = tf.convert_to_tensor(test_mask, dtype=tf.bool)

>   - **50/50 train-test split** is done randomly over all nodes, not per scene.
>   - Data is split **at the node level**: features, labels, and valid masks are all subset accordingly.
>   - Tensors are briefly converted to NumPy for indexing, then **converted back to TensorFlow tensors**.
>   - `train_mask` and `test_mask` will be used to compute the loss **only for nodes with valid labels**.

*Display*:

In [ ]:
print("Shape of training features:", train_node_features.shape)
print("Shape of training labels:", train_labels.shape)
print("Shape of training masks:", train_mask.shape)

print("Shape of test features:", test_node_features.shape)
print("Shape of test labels:", test_labels.shape)
print("Shape of test masks:", test_mask.shape)

#### Graph Attention Layer Definition (GAT Layer)

In [ ]:
class GraphAttention(layers.Layer):
    def __init__(
        self,
        units,
        kernel_initializer="glorot_uniform",
        kernel_regularizer=None,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.units = units
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
    
    def build(self, input_shape):
        # Learnable linear projection for input features
        self.kernel = self.add_weight(
            shape=(input_shape[0][-1], self.units),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            name="kernel",
            trainable=True,
        )
        # Attention kernel for computing attention scores [h_i || h_j] -> score
        self.kernel_attention = self.add_weight(
            shape=(self.units * 2, 1),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            name="kernel_attention",
            trainable=True,
        )
        super().build(input_shape)
        
    def call(self, inputs):
        node_states, edges = inputs

        # Step 1: Linear transformation of node features
        h = tf.matmul(node_states, self.kernel)  # (N, units)

        # Step 2: Compute attention scores for each edge [h_i || h_j]
        edge_states = tf.gather(h, edges)   # (E, 2, units)
        edge_states = tf.reshape(edge_states, (tf.shape(edges)[0], 2 * self.units))
        scores = tf.nn.leaky_relu(tf.matmul(edge_states, self.kernel_attention))    # (E,1)
        scores = tf.squeeze(scores, -1)     # (E,)
        
        # Step 3: Normalize attention scores (softmax-like) per target node
        scores_exp = tf.exp(tf.clip_by_value(scores, -2, 2))    # for stability
        denom = tf.math.unsorted_segment_sum(
            data=scores_exp,
            segment_ids=edges[:, 0],    # group by target node
            num_segments=tf.shape(node_states)[0]
        )
        # broadcast denom back to each edge
        denom_per_edge = tf.gather(denom, edges[:, 0])
        alpha = scores_exp / (denom_per_edge + tf.keras.backend.epsilon())
        
        # Step 4: Weighted sum of neighbor features using attention scores
        neigh = tf.gather(h, edges[:, 1])   # (E, units)
        out = tf.math.unsorted_segment_sum(
            data=neigh * alpha[:, tf.newaxis],
            segment_ids=edges[:, 0],    # group by target node
            num_segments=tf.shape(node_states)[0]
        )
        return out

>   - This class defines a custom GAT layer (Graph Attention Layer).  
>   - It follows the original GAT paper pipeline:  
>       1. **Linear transformation** of node features: $h_i = W \cdot x_i$  
>       2. **Attention score** computation for each edge using concatenated features $[h_i||h_j]$  
>       3. **Softmax-style normalization** across all neighbors of each target node  
>       4. Weighted aggregation of neighbor states using attention weights $\alpha_{ij}$  
>   - `unsorted_segment_sum` is key: it groups and sums messages **per target node**.  
>   - The graph is **directed in edges**, but the dataset was made undirected earlier (so info still flows both ways).

#### Multi-Head Graph Attention Layer Definition

In [ ]:
class MultiHeadGraphAttention(layers.Layer):
    def __init__(self, units, num_heads=8, merge_type="concat", **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.merge_type = merge_type
        # Initialize multiple GAT heads
        self.attention_heads = [GraphAttention(units) for _ in range(num_heads)]

    def call(self, inputs):
        node_states, edges = inputs

        # Apply each attention head independently
        head_outputs = [head([node_states, edges]) for head in self.attention_heads]

        # Merge head outputs: either concatenate or average
        if self.merge_type == "concat":
            h = tf.concat(head_outputs, axis=-1)    # Shape: (N, units * num_heads)
        else:
            h = tf.reduce_mean(tf.stack(head_outputs, axis=-1), axis=-1)    # Shape: (N, units)

        # Apply activation function
        return tf.nn.relu(h)

>   - **Multi-head attention**: several `GraphAttention` layers (heads) run in parallel.  
>   - Each head can focus on **different neighbors or patterns**, like different “views” of the graph.
>   - Two ways to merge the heads:  
>       - `"concat"`: concatenate all head outputs (like in the original GAT paper) $\to$ richer representation.  
>       - `"average"`: average across heads $\to$ more compact, less parameters.
>   - `tf.nn.relu` is applied after merging to introduce non-linearity.

#### Graph Attention Network Model Definition

In [ ]:
class GraphAttentionNetwork(keras.Model):
    def __init__(
        self,
        node_features,
        edges,
        hidden_units=32,
        num_heads=4,
        num_layers=2,
        output_dim=2,
        **kwargs
    ):
        super().__init__(**kwargs)
        # Store fixed graph structure
        self.node_features = node_features
        self.edges = edges

        # Initial transformation of input features
        self.preprocess = layers.Dense(hidden_units * num_heads, activation="relu")

        # Stack multiple multi-head GAT layers
        self.gat_layers = [
            MultiHeadGraphAttention(hidden_units, num_heads, merge_type="concat")
            for _ in range(num_layers)
        ]

        # Output layer: regresses to (x, y)
        self.output_layer = layers.Dense(output_dim)

    def call(self, inputs):
        x, edges = inputs

        # Initial feature transformation
        x = self.preprocess(x)

        # Apply GAT layers with residual connections
        for gat in self.gat_layers:
            x = gat([x, edges]) + x
        # Final output layer for (x, y) coordinates
        return self.output_layer(x)

    def train_step(self, data):
        indices, y_true = data

        # Custom training loop with batch sampling
        with tf.GradientTape() as tape:
            y_pred = self([self.node_features, self.edges])
            y_pred_batch = tf.gather(y_pred, indices)   # Select batch samples
            loss = self.compiled_loss(y_true, y_pred_batch)

        # Compute gradients and update weights
        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        # Track metrics
        self.compiled_metrics.update_state(y_true, y_pred_batch)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        indices, y_true = data
        y_pred = self([self.node_features, self.edges])
        y_pred_batch = tf.gather(y_pred, indices)
        loss = self.compiled_loss(y_true, y_pred_batch)
        self.compiled_metrics.update_state(y_true, y_pred_batch)
        return {m.name: m.result() for m in self.metrics}

    def predict_step(self, data):
        indices = data
        y_pred = self([self.node_features, self.edges])
        return tf.gather(y_pred, indices)

>   - `GraphAttentionNetwork` is a **custom Keras model** combining:
>       - `Dense` layer for initial embedding.
>       - A **stack of Multi-Head GAT layers** for message passing.
>       - A final `Dense` layer for **regression to 2D output** (`(x, y)`).
>   - The `call()` method does a forward pass through the model.
>   - `train_step`, `test_step`, and `predict_step` are overridden to allow training/prediction using **node indices**, while the full graph stays in memory.
>   - `+ x` in GAT loop $\to$ **residual connection**, helps **gradient flow** and **stabilizes learning**.

#### Model Training and Evaluation

In [ ]:
# ----------- Hyperparameters -----------
HIDDEN_UNITS = 32       # size of each GAT attention head
NUM_HEADS = 4           # number of parallel attention heads
NUM_LAYERS = 2          # number of stacked GAT layers
OUTPUT_DIM = 2          # we predict 2 values: (future_x, future_y)

NUM_EPOCHS = 100
BATCH_SIZE = 256
VALIDATION_SPLIT = 0.1  # 10% of train data used for validation
LEARNING_RATE = 1e-3
PATIENCE = 10           # early stopping patience

# ----------- Loss and optimizer -----------
loss_fn = keras.losses.MeanSquaredError()                   # for regression
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
mae_metric = keras.metrics.MeanAbsoluteError(name="mae")    # easier to interpret

# ----------- Early stopping to prevent overfitting -----------
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_mae",          # monitor validation MAE
    min_delta=1e-4,             # minimal change to be considered improvement
    patience=PATIENCE,          # stop if no improvement over X epochs
    restore_best_weights=True
)

# ----------- Instantiate and compile the GAT model -----------
gat_model = GraphAttentionNetwork(
    node_features=node_features,
    edges=edges,
    hidden_units=HIDDEN_UNITS,
    num_heads=NUM_HEADS,
    num_layers=NUM_LAYERS,
    output_dim=OUTPUT_DIM
)


gat_model.compile(
    loss=loss_fn,
    optimizer=optimizer,
    metrics=[mae_metric]
)

# ----------- Train the model -----------
gat_model.fit(
    x=train_indices,
    y=train_labels,
    validation_split=VALIDATION_SPLIT,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    callbacks=[early_stopping],
    verbose=2,
)

# ----------- Evaluate on test set -----------
loss, mae = gat_model.evaluate(
    x=test_indices,
    y=test_labels,
    verbose=0
)

>   - **Model hyperparameters** (`units`, `heads`, `layers`) define the GAT architecture capacity.
>   - `MeanSquaredError` is the loss function used for regression (sensitive to outliers).
>   - `MeanAbsoluteError` is used as a metric (more interpretable, in same unit as coordinates).
>   - **EarlyStopping** helps prevent overfitting by monitoring `val_mae`.
>   - The model is trained with:
>       - full graph fixed (`node_features`, `edges`)
>       - mini-batch sampling using node indices (`train_indices`)
>       - training targets being future coordinates
>   - The evaluation returns the final **test loss and MAE**, which reflect prediction performance.

*Display*:

In [ ]:
print(f"\nTest MAE: {mae:.4f}")

#### Prediction

In [ ]:
# Predict future (x, y) positions for the test nodes
# The model uses the fixed graph (features + edges) and returns predictions for selected indices
test_preds = gat_model.predict(x=test_indices)

>   - `gat_model.predict(x=test_indices)` returns the model's prediction for each node in `test_indices`.
>   - The model uses the **fixed full graph structure** (defined during instantiation).
>   - Only the **nodes corresponding to** `test_indices` are extracted from the prediction output using `predict_step`.
>   - The output `test_preds` is a tensor of shape `(num_test_nodes, 2)` $\to$ each row contains the predicted `(x, y)` future position.

*Display*:

In [ ]:
# Display the first 10 predictions with errors
errors = []
for i, (pred, true) in enumerate(zip(test_preds[:], test_labels[:])):
    true_norm = tf.norm(true).numpy()
    l2_error = tf.norm(pred - true).numpy()
    errors.append({
        "index": i,
        "prediction": pred,
        "ground_truth": true,
        "l2_error": l2_error
    })

errors_sorted = sorted(errors, key=lambda x: x["l2_error"], reverse=False)

for i, item in enumerate(errors_sorted[:]):
    pred = item["prediction"]
    true = item["ground_truth"]
    l2_error = item["l2_error"]
    print(f"Example {i+1}:")
    print(f"\tPrediction   = ({pred[0]:.2f}, {pred[1]:.2f})")
    print(f"\tGround Truth = ({true[0]:.2f}, {true[1]:.2f})")
    print(f"\tL2 Error     = {l2_error:.2f} units")
    print("---" * 20)